In [1]:
import os
import os.path as osp
from glob import glob
from typing import List, Dict, Union

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных

In [2]:
os.makedirs('data/', exist_ok=True)

In [ ]:
!unzip /content/ИТЭЛМА_ЛЦТ.zip -d data/

In [3]:
DATASET_DIR = "/content/data/ИТЭЛМА_ЛЦТ"

# Обработка метаданных

## Приведение к одному виду, исправление названий столбцов колонок

In [4]:
def preprocess_xlsx(df_path: str, header: int = 0) -> pd.DataFrame:
    df = pd.read_excel(df_path, header=header)
    df.columns = df.columns.str.lower()

    unnamed_cols = [f"unnamed: {i}" for i in range(3,7)]
    unnamed_cols.insert(0, 'газы крови')

    blood_gases_names = list(df.iloc[0][2:].values)
    unnamed_mapping = {k: v for k,v in zip(unnamed_cols, blood_gases_names)}
    df = df.rename(columns=unnamed_mapping)

    return df.iloc[1:]

In [5]:
pd.read_excel('/content/data/ИТЭЛМА_ЛЦТ/hypoxia.xlsx').head() # до

,folder_id,Диагноз,Газы крови,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,Ph,CO2,Glu,LAC,BE,NaN
1,2,Беременность 40-41 неделя. \nГоловное предлежа...,NaN,NaN,NaN,NaN,NaN,NaN
2,12,I своевременные оперативные роды\nОстрая гипок...,–,-,–,–,–,NaN
3,13,I своевременные оперативные роды\nОстрая гипок...,–,-,–,–,–,NaN
4,22,II своевременные роды.\nРубец на матке после о...,–,-,–,–,–,NaN


In [6]:
hypoxia_df = preprocess_xlsx('/content/data/ИТЭЛМА_ЛЦТ/hypoxia.xlsx', header=0)

In [7]:
hypoxia_df.head() # после (последняя колонка фантомна, нужно дропать ручками)

,folder_id,диагноз,Ph,CO2,Glu,LAC,BE,unnamed: 7
1,2,Беременность 40-41 неделя. \nГоловное предлежа...,NaN,NaN,NaN,NaN,NaN,NaN
2,12,I своевременные оперативные роды\nОстрая гипок...,–,-,–,–,–,NaN
3,13,I своевременные оперативные роды\nОстрая гипок...,–,-,–,–,–,NaN
4,22,II своевременные роды.\nРубец на матке после о...,–,-,–,–,–,NaN
5,16,I своевременные роды в 35 лет\nОстрая гипоксия...,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
hypoxia_df = hypoxia_df.drop('unnamed: 7', axis=1)

In [9]:
regular_df = preprocess_xlsx('/content/regular_fixed.xlsx', header=1)

In [10]:
regular_df.head(10) # после обработки

,folder_id,диагноз,Ph,CO2,Glu,LAC,BE
1,"155,156",I своевременные оперативные роды в чисто ягоди...,–,–,–,–,–
2,150,II своевременные оперативные роды\nРубец на ма...,–,–,–,–,–
3,139,V своевременные роды крупным плодом\nРаннее из...,–,–,–,–,–
4,134,I своевременные оперативные роды в 33 года на ...,–,–,–,–,–
5,"128,129","II своевременные роды\nГСД, Инсулинотерапия",7.397,26.9,5.4,4.1,-7.7
6,"114,115",II своевременные оперативные роды\nРубец на ма...,–,–,–,–,–
7,112,II своевременные оперативные роды\nПатология п...,7.419,26.1,3.7,1.7,-7
8,5,III своевременные роды\nМиопия высокой степени...,–,–,–,–,–
9,"48,49",II своевременные оперативные роды \nРубец на м...,–,–,–,–,–
10,53,I своевременные оперативные роды в 37 лет\nI п...,7.5,17.3,5.4,2.1,-9.2


## Разбираемся с folder_id

In [11]:
def extract_folder_ids(df: pd.DataFrame) -> List[int]:
    """
    Возвращает список все folder_id из датафрейма,
    т.е использует информацию из метаданных - директорий на самом деле может не существовать
    """
    folder_ids = []
    for item in list(df.folder_id.astype(str).values):
       folder_ids.extend(str(x) for x in item.replace('.', ',').split(','))
    return sorted(folder_ids)

In [12]:
# убираем строки с пустыми id-шниками директорий
hypoxia_df = hypoxia_df[hypoxia_df['folder_id'].notna() & (hypoxia_df['folder_id'] != '')]
regular_df = regular_df[regular_df['folder_id'].notna() & (regular_df['folder_id'] != '')]

In [13]:
regular_ids = extract_folder_ids(regular_df) # id из метадаты

In [14]:
hypoxia_ids = extract_folder_ids(hypoxia_df) # id из метадаты

In [15]:
def get_folder_ids(target_class: str, dataset_dir: str = DATASET_DIR):
    """
    Возвращает реально существующие под-директории для таргетного класса класса,
    т.е по пути dataset_dir/target_class
    """
    target_class_dir = osp.join(dataset_dir, target_class)
    return sorted([str(i) for i in os.listdir(target_class_dir) if osp.isdir(osp.join(target_class_dir, i))])


In [16]:
gt_hypoxia_folders = get_folder_ids('hypoxia') # реально существуют
gt_regular_folders = get_folder_ids('regular') # реально существуют

In [17]:
print(f"ID для которых отсутствуют данные (hypoxia): {[i for i in hypoxia_ids if i not in gt_hypoxia_folders]}")
print(f"ID для которых отсутствуют данные (regular): {[i for i in regular_ids if i not in gt_regular_folders]}")

ID для которых отсутствуют данные (hypoxia): ['15', '26']
ID для которых отсутствуют данные (regular): ['118', '126', '41', '47', '48', '49', '52', '57']


In [23]:
def separate_folder_ids(
    df: pd.DataFrame,
) -> pd.DataFrame:
    result_df = df.copy()
    result_df["folder_id"] = (
        result_df["folder_id"]
        .astype(str)
        .str.strip()
        .str.replace('.', ',')
        .str.split(",")
    )
    result_df = result_df.explode("folder_id", ignore_index=True)
    result_df["folder_id"] = result_df["folder_id"].str.strip()

    return result_df


In [25]:
hypoxia_df_final = separate_folder_ids(hypoxia_df)

In [26]:
regular_df_final = separate_folder_ids(regular_df)

In [27]:
regular_df_final

,folder_id,диагноз,Ph,CO2,Glu,LAC,BE
0,155,I своевременные оперативные роды в чисто ягоди...,–,–,–,–,–
1,156,I своевременные оперативные роды в чисто ягоди...,–,–,–,–,–
2,150,II своевременные оперативные роды\nРубец на ма...,–,–,–,–,–
3,139,V своевременные роды крупным плодом\nРаннее из...,–,–,–,–,–
4,134,I своевременные оперативные роды в 33 года на ...,–,–,–,–,–
...,...,...,...,...,...,...,...
101,117,I своевременные оперативные роды в 36 лет\nПер...,7.37,35.3,3.7,1.7,-4.5
102,118,I своевременные оперативные роды в 36 лет\nПер...,7.37,35.3,3.7,1.7,-4.5
103,108,I своевременные оперативные роды в 37 лет\nСла...,NaN,NaN,NaN,NaN,NaN
104,109,I своевременные оперативные роды в 37 лет\nСла...,NaN,NaN,NaN,NaN,NaN


## Выгружаем обработанные метаданные в csv

In [29]:
regular_df_final.to_csv('regular_preprocessed.csv')

In [30]:
hypoxia_df_final.to_csv('hypoxia_preprocessed.csv')

#